In [20]:
# open a file in read mode
def openfile(filename):
    file=open(filename,"r")
    data=file.read()
    return data

In [21]:
# read a file content and split the data in lines seperated by comma into list
def filecontent(file):
    lines=file.split("\n")
    #print(lines)
    lists=[]
    for line in lines:
        lists.append(line.split(","))
    
    #print(lists)
    return lists

In [23]:
# use items and basket file after preprocing to convert into list

#file1='./basket1.csv'
#file2='./items1.csv'
file1='./basket.csv'
file2='./items.csv'
data1=openfile(file1)
data2=openfile(file2)
items_list= filecontent(data1)
basket_list=filecontent(data2)
    


In [5]:
# apriori algorithm

import argparse
from itertools import chain, combinations


def joinset(itemset, k):
    return set([i.union(j) for i in itemset for j in itemset if len(i.union(j)) == k])


def subsets(itemset):
    return chain(*[combinations(itemset, i + 1) for i, a in enumerate(itemset)])
    

def itemset_from_data(data):
    itemset = set()
    transaction_list = list()
    for row in data:
        transaction_list.append(frozenset(row))
        for item in row:
            if item:
                itemset.add(frozenset([item]))
    return itemset, transaction_list


def itemset_support(transaction_list, itemset, min_support=0):
    len_transaction_list = len(transaction_list)
    l = [
        (item, float(sum(1 for row in transaction_list if item.issubset(row)))/len_transaction_list) 
        for item in itemset
    ]
    return dict([(item, support) for item, support in l if support >= min_support])


def freq_itemset(transaction_list, c_itemset, min_support):
    f_itemset = dict()

    k = 1
    while True:
        if k > 1:
            c_itemset = joinset(l_itemset, k)
        l_itemset = itemset_support(transaction_list, c_itemset, min_support)
        if not l_itemset:
            break
        f_itemset.update(l_itemset)
        k += 1

    return f_itemset    


def apriori(data, min_support, min_confidence):
    # Get first itemset and transactions
    itemset, transaction_list = itemset_from_data(data)

    # Get the frequent itemset
    f_itemset = freq_itemset(transaction_list, itemset, min_support)

    # Association rules
    rules = list()
    """for item, support in list(f_itemset.items()):
        if len(item) > 1:
            for A in subsets(item):
                B = item.difference(A)
                if B:
                    A = frozenset(A)
                    AB = A | B
                    confidence = float(f_itemset[AB]) / f_itemset[A]
                    if confidence >= min_confidence:
                        rules.append((A, B, confidence))  """  
    return rules, f_itemset


def print_report(rules, f_itemset):
    print('--Frequent Itemset--')
    Frequent_list=[]
    for item, support in sorted(list(f_itemset.items()), key=lambda item_support: item_support[1]):
        print(('[I] {} : {}'.format(tuple(item), round(support, 4))))
        Frequent_list.append(('[I] {} : {}'.format(tuple(item), round(support, 4))))
        

    print('')
    print('--Rules--')
    for A, B, confidence in sorted(rules, key=lambda A_B_confidence: A_B_confidence[2]):
        print(('[R] {} => {} : {}'.format(tuple(A), tuple(B), round(confidence, 4)))) 
    return Frequent_list

def data_from_csv(filename):
    f = open(filename, 'rU')
    for l in f:
        row = list(map(str.strip, l.split(',')))
        yield row


def parse_options():
    optparser = argparse.ArgumentParser(description='Apriori Algorithm.')
    optparser.add_argument(
        '-f', '--input_file',
        dest='filename',
        help='filename containing csv',
        required=True
    )
    optparser.add_argument(
        '-s', '--min_support',
        dest='min_support',
        help='minimum support',
        default=0.25,
        type=float
    )
    optparser.add_argument(
        '-c', '--min_confidence',
        dest='min_confidence',
        help='minimum confidence',
        default=0.5,
        type=float
    )
    return optparser.parse_args()


def main():
    options = parse_options()

    data = data_from_csv(options.filename)
    rules, itemset = apriori(data, options.min_support, options.min_confidence)
    print_report(rules, itemset)



In [6]:
#create a MAG by checking the subset from the output list of apriori algorithm and basket with symptoms and disease
def create_MAG(inputf,data):
    
    MAG=[]
    inputf=list(map(set,inputf))
    for pattern in inputf: 
        itemset=[]
        basket=[]
        for item in data:
            
            #print("item",inputf[:len(item)-1])
            #print("patterns",pattern)
            if pattern .issubset(item[:len(item)-1]):
                
                if item [len(item)-1] not in basket:
                   
                    basket.append(item[len(item)-1])
                    #print("basket",basket)
        itemset.append((pattern))
                
        itemset.append((basket))
        if itemset not in MAG:
            MAG.append(list(map(set,itemset)))
           # print(MAG)
    #print("itemset",list(map(set,itemset)))
   # print("MAG: ",MAG)
    #for c in MAG:
       # print("MAG:",[list(x) for x in c])
    return MAG


In [24]:
import numpy 
import pandas as pd
# call apriori algorithm with items_list which contains symptoms with minimum support of 0.02 and confidence with 0.8
rules, f_itemset=list(apriori(items_list,0.02,0.8))
print(len(f_itemset),len(rules))
out=[]
for item in f_itemset:
    out.append(list (item))

final_list=[]
for item in rules:
    #print(list (item))
    rules_list=[]
    for each in item[:len(item)-1]:
        a=(list (each))
        
        rules_list.append(a)
    rules_list.append(item[len(item)-1])
    final_list.append(rules_list)

    
MAG=create_MAG(out,basket_list) 

print(MAG[0])
node=[]
#create of name for MAG
for i in range(0,len(MAG)):
    node.append('MAG{}'.format(i))
    
final_data=[]
# convert MAG into list of node,Symptoms and Diagnosis
for i in range(0,len(node)):
    new_row=[]    
    new_row.append(node[i])
    for item in MAG[i]:
        row=[each for each in item]
        
        new_row.append(row) 
        
        
    final_data.append(new_row)        
    
    
MAG1=pd.DataFrame(final_data)
MAG1.columns=["Node","Symptoms","Diagnosis"]
print(MAG1.head)
MAG1.to_csv("./final_output.csv",index=False)
rules_df=pd.DataFrame(final_list)
rules_df.to_csv("./rules.csv",index=False)
MAG1
    

652 0
[{'Incontinence (leaking urine)'}, {'Cerebral vascular accident\x0bstroke', 'Multiple sclerosis\x0bMS', 'Diabetes mellitus type 2\x0belevated blood sugar', '" Jantoven overdose"""'}]
<bound method NDFrame.head of        Node                                           Symptoms  \
0      MAG0                     [Incontinence (leaking urine)]   
1      MAG1                                    [Blurry vision]   
2      MAG2                                  [Visual problems]   
3      MAG3                            [Eye pain (Irritation)]   
4      MAG4                           [Visual flashing lights]   
5      MAG5                          [Vertigo (Room spinning)]   
6      MAG6                                         [Numbness]   
7      MAG7                                        [Dizziness]   
8      MAG8                                     [Foot itching]   
9      MAG9                            [Headache and weakness]   
10    MAG10                                     [Eye sw

,Node,Symptoms,Diagnosis
0,MAG0,[Incontinence (leaking urine)],"[Cerebral vascular accident stroke, Multiple s..."
1,MAG1,[Blurry vision],"[Corneal abrasion scrape on eye, Orbital cellu..."
2,MAG2,[Visual problems],"[Cerebral contusion bruise of brain, Diabetic ..."
3,MAG3,[Eye pain (Irritation)],"[Gilbert's disease liver abnormality, Lacunar ..."
4,MAG4,[Visual flashing lights],"[Newborn jaundice newborn skin discoloration, ..."
5,MAG5,[Vertigo (Room spinning)],"[Brain tumor cancer of the brain, Trauma, Basi..."
6,MAG6,[Numbness],"[Cerebral vascular accident stroke, Gallbladde..."
7,MAG7,[Dizziness],"[Brain tumor cancer of the brain, Cerebral vas..."
8,MAG8,[Foot itching],[Vertebral- basilar artery insufficiency inade...
9,MAG9,[Headache and weakness],"[Cerebral contusion bruise of brain, Cerebral ..."


In [25]:
final_df=pd.read_csv('./final_output.csv')
final_df_list=[]
for index,row in final_df.iterrows():
    temp_list=[]
    #print(row['Diagnosis'].strip('[]').split(','),len(row['Diagnosis'].strip('[]').split(',')))
    temp_list.append(row['Node'])
    temp_list.append(row['Symptoms'])
    temp_list.append(row['Diagnosis'])
    temp_list.append(len(row['Diagnosis'].strip('[]').split(',')))
    temp_list.append(len(row['Symptoms'].strip('[]').split(',')))
    final_df_list.append(temp_list)

file_data=pd.DataFrame(final_df_list)

file_data.columns=["Node","Symptoms","Diagnosis","Diagnosis_Count","Symptoms_Count"]
file_data.to_csv("./final_list_output.csv",index=False)

In [26]:
import pandas as pd
df=pd.read_csv('./final_output.csv')

df['Symptoms']=df.Symptoms.apply(lambda x: x[1:-1].split(','))
bucket={}
bucket_list=[]
for index, row in df.iterrows(): 
    new_key=(len(row['Symptoms']))
    bucket.setdefault(new_key,[]).append(row)
    
        #print((bucket))

#print(bucket[8])               
write_list=[]
for each in bucket.keys():
    bucket_element=[]
    for item in bucket[each]:
        
        bucket_element.append(set(item['Symptoms']))
    write_list.append(bucket_element)
    
   # writefile(write_list,file)
data=pd.DataFrame(write_list)
data.to_csv('out.csv', index=False, header=False)
#print(write_list)
new_list_item={}
for index in range(1,9):
    for each in write_list:
        for item in each:
            new_key=len(item)
            for subitem in item:
                new_list_item.setdefault(new_key,[]).append((subitem))
         


In [16]:
#print(new_list_item[1])
#print(new_list_item[1])
A_intersection=[]
for index in range(2,9): 
    u = set.intersection(set(new_list_item[1]),set(new_list_item[index]))
    A_intersection.append(u)
#print(A_intersection)
final_intersect=list(set().union(*A_intersection))
u = set.difference(set(new_list_item[1]),set(final_intersect))
print(len(u))
final=(list(set().union(set(new_list_item[1]),set(new_list_item[2]),set(new_list_item[3]),set(new_list_item[4]),set(new_list_item[5]),set(new_list_item[6]),set(new_list_item[7]),set(new_list_item[8]))))

KeyError: 2

In [15]:
import pandas as pd
new_sym_dis_matrix=pd.read_csv("new_transformed_dataset.csv")
#print(set(new_sym_dis_matrix['S']))
#print(len(set.intersection(set(final),new_sym_dis_matrix['S'])))
#print((set(new_sym_dis_matrix['S'])))
final= [w.replace(' ', '').replace("'",'') for w in final]
print((set.difference(set(new_sym_dis_matrix['S']),set(final))))

NameError: name 'final' is not defined

In [27]:
def writefile(lists,filename):
    import csv
    import os
        #writer = csv.DictWriter(file)
    writer=csv.writer(file)
    writer.writerows(lists)
    

In [29]:
# find the relationship between MAGS 
relation_list=[]
Max=0
for each in bucket.keys():
    if each>Max:
        Max=each
file=open("relation.csv","w") 

for each in range(1,Max):
    for preceding_item in bucket[each]:
        
        for item in bucket[each+1]:
            
            A=set(preceding_item['Symptoms'])
            B= set(item['Symptoms'])
            if A.issubset(B):
                concat_set=[]
            
                concat_set.append(list(A))
                concat_set.append(list(B))
                concat_set.append(preceding_item['Node'])
                concat_set.append(item['Node'])
                concat_set.append(list(B.difference(A)))
               
                relation_list.append(concat_set)
                #writefile(concat_set,"relation.csv")
#print(relation_list)
#writefile(relation_list,"relation.csv")

data=pd.DataFrame(relation_list)

for i in data.columns:
    data[i]=([(str(x).strip('[]').replace('"','').replace(' ','')) for x in  data[i]])

data.columns=["From","To","Initial","Final","Relation"]
data.to_csv("relation.csv", index=False)


In [ ]:
print(data.head())

In [ ]:
import pandas as pd
file=pd.read_csv('/home/dev/Desktop/sdsort/sym_dis_matrix.csv')
file